## Collaborative Filtering

*Prepared by:*
**Jude Michael Teves**  
Faculty, Software Technology Department  
College of Computer Studies - De La Salle University

This notebook shows how to perform a collaborative filtering type of recommender system.

## Preliminaries

### Import libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from scipy.stats import pearsonr

### Load Data

We will be using the MovieLens dataset here. I have already preprocessed the data so it will be easier for us to process later on.

In [2]:
df_ratings = pd.read_csv('https://raw.githubusercontent.com/Cyntwikip/data-repository/main/movielens_movie_ratings.csv')
df_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [3]:
df_genres = pd.read_csv('https://raw.githubusercontent.com/Cyntwikip/data-repository/main/movielens_movie_genres.csv')
df_genres.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## User-based Collaborative Filtering

### Build User-Item Matrix

In [4]:
user_id = 3

In [5]:
df_user = df_ratings.pivot(index='userId', columns='movieId', values='rating')
df_user

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Retrieve *k* most similar users

#### Preprocessing - Mean Imputation

In [6]:
df_user_filled = df_user.apply(lambda x: x.fillna(x.mean()), axis=1)
df_user_filled.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.366379,4.000000,4.366379,4.366379,4.000000,4.366379,4.366379,4.366379,4.366379,...,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379
2,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,...,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276
3,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,...,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556
5,4.000000,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,...,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364


#### Similarity Computation

In [7]:
k = 10
reference_user = df_user_filled.loc[user_id]
user_similarities = df_user_filled.apply(lambda x: pearsonr(x, reference_user)[0], axis=1)
similar_users = user_similarities.drop(user_id, axis=0).nlargest(k)
similar_users

C:\Users\jude_\anaconda3\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


userId
441    0.117418
496    0.067878
549    0.064006
231    0.061159
527    0.058456
537    0.058072
313    0.055313
518    0.050288
244    0.049511
246    0.048314
dtype: float64

### Get average rating of similar users

In [8]:
predicted_ratings = df_user.loc[similar_users.index].mean().sort_values(ascending=False)
predicted_ratings

movieId
2450      5.0
68954     5.0
68486     5.0
2683      5.0
1199      5.0
         ... 
193581    NaN
193583    NaN
193585    NaN
193587    NaN
193609    NaN
Length: 9724, dtype: float64

#### Recommend items

In [9]:
user_unrated_items = df_user.loc[user_id].isna()
recommended_items = predicted_ratings[user_unrated_items].head(10)
recommended_items

movieId
2450     5.0
68954    5.0
68486    5.0
2683     5.0
1199     5.0
1200     5.0
1997     5.0
3153     5.0
66371    5.0
1213     5.0
dtype: float64

Let's observe how other similar users rated those items.

In [10]:
df_user.loc[similar_users.index, recommended_items.index]

movieId,2450,68954,68486,2683,1199,1200,1997,3153,66371,1213
userId,,,,,,,,,,
441,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
527,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,NaN,5.0
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN


### Variation: Get weighted average of similar users

In [11]:
def get_weighted_similarity(x):
    weighted_similarity = x*similar_users
    norm = similar_users[~weighted_similarity.isna()].sum()
    rating = weighted_similarity.sum()/norm
    return rating

predicted_ratings = df_user.loc[similar_users.index].apply(get_weighted_similarity, axis=0)
predicted_ratings = predicted_ratings.sort_values(ascending=False)
predicted_ratings

<ipython-input-11-8c1860f1db0e>:4: RuntimeWarning: invalid value encountered in double_scalars
  rating = weighted_similarity.sum()/norm


movieId
1333      5.0
1982      5.0
3071      5.0
1961      5.0
2450      5.0
         ... 
193581    NaN
193583    NaN
193585    NaN
193587    NaN
193609    NaN
Length: 9724, dtype: float64

#### Recommend items

In [12]:
user_unrated_items = df_user.loc[user_id].isna()
recommended_items = predicted_ratings[user_unrated_items].head(10)
recommended_items

movieId
1333    5.0
1982    5.0
3071    5.0
1961    5.0
2450    5.0
2118    5.0
2355    5.0
2137    5.0
4638    5.0
1035    5.0
dtype: float64

Let's observe how other similar users rated those items.

In [13]:
df_user.loc[similar_users.index, recommended_items.index]

movieId,1333,1982,3071,1961,2450,2118,2355,2137,4638,1035
userId,,,,,,,,,,
441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Item-based Collaborative Filtering

### Build Item-User Matrix

In [14]:
user_id = 3
item_id = 1

In [15]:
df_item = df_ratings.pivot(index='movieId', columns='userId', values='rating')
df_item

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Retrieve *k* most similar items

#### Preprocessing - Mean Imputation

In [16]:
df_item_filled = df_item.apply(lambda x: x.fillna(x.mean()), axis=1)
df_item_filled.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,3.920930,3.920930,3.920930,4.000000,3.92093,4.500000,3.920930,3.920930,3.920930,...,4.000000,3.920930,4.000000,3.000000,4.000000,2.500000,4.000000,2.500000,3.000000,5.000000
2,3.431818,3.431818,3.431818,3.431818,3.431818,4.00000,3.431818,4.000000,3.431818,3.431818,...,3.431818,4.000000,3.431818,5.000000,3.500000,3.431818,3.431818,2.000000,3.431818,3.431818
3,4.000000,3.259615,3.259615,3.259615,3.259615,5.00000,3.259615,3.259615,3.259615,3.259615,...,3.259615,3.259615,3.259615,3.259615,3.259615,3.259615,3.259615,2.000000,3.259615,3.259615
4,2.357143,2.357143,2.357143,2.357143,2.357143,3.00000,2.357143,2.357143,2.357143,2.357143,...,2.357143,2.357143,2.357143,2.357143,2.357143,2.357143,2.357143,2.357143,2.357143,2.357143
5,3.071429,3.071429,3.071429,3.071429,3.071429,5.00000,3.071429,3.071429,3.071429,3.071429,...,3.071429,3.071429,3.071429,3.000000,3.071429,3.071429,3.071429,3.071429,3.071429,3.071429


#### Similarity Computation

In [17]:
k = 5
reference_item = df_item_filled.loc[item_id]
item_similarities = df_item_filled.apply(lambda x: pearsonr(x, reference_item)[0], axis=1)
user_rated_items = df_item.loc[:, user_id].dropna().index.tolist()
item_similarities = item_similarities.drop(item_id, axis=0).loc[user_rated_items]
similar_items = item_similarities.nlargest(k)
similar_items

C:\Users\jude_\anaconda3\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


movieId
1275    0.112876
2080    0.112139
2424    0.110641
688     0.092266
2288    0.081103
dtype: float64

### Get weighted average of similar items

This is how `user_id = 3` will rate `movieId = 1`

In [18]:
df_similar_items = df_item.loc[similar_items.index, user_id]
df_similar_items

movieId
1275    3.5
2080    0.5
2424    0.5
688     0.5
2288    4.0
Name: 3, dtype: float64

In [19]:
def get_item_weighted_similarity(x):
    weighted_similarity = x*similar_items
    norm = similar_items[~weighted_similarity.isna()].sum()
    rating = weighted_similarity.sum()/norm
    return rating

get_item_weighted_similarity(df_similar_items)

1.7229008908010734

## References

1. F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. https://doi.org/10.1145/2827872

## End
<sup>made by **Jude Michael Teves**</sup> <br>
<sup>for comments, corrections, suggestions, please email:</sup><sup> <href>judemichaelteves@gmail.com</href> or <href>jude.teves@dlsu.edu.ph</href></sup><br>